# 色温度変換をしてみる

## 参考サイト
[Bradford 変換 (白色点の変換)](http://w3.kcua.ac.jp/~fujiwara/infosci/colorspace/bradford.html)

[色温度のxy色度(英語版Wikipediaより)](https://en.wikipedia.org/wiki/Standard_illuminant#White_points_of_standard_illuminants)

In [1]:
import os
import sys
import cv2
import numpy as np
from scipy import linalg
import matplotlib.pyplot as plt
%matplotlib inline

const_xyz_to_lms = [[0.8951000, 0.2664000, -0.1614000],
                    [-0.7502000, 1.7135000, 0.0367000],
                    [0.0389000, -0.0685000, 1.0296000]]
const_d65_xy = [0.31271, 0.32902]
const_d50_xy = [0.34567, 0.35850]

def get_white_point_conv_matrix(src=const_d65_xy, dst=const_d50_xy):
    """
    参考： http://w3.kcua.ac.jp/~fujiwara/infosci/colorspace/bradford.html
    """
    if len(src) == 2:
        src = [src[0], src[1], 1 - (src[0] + src[1])]
    if len(dst) == 2:
        dst = [dst[0], dst[1], 1 - (dst[0] + dst[1])]

    src = np.array(src)
    dst = np.array(dst)

    src = src / src[1]
    dst = dst / dst[1]

    # LMS値を求めよう
    # --------------------------------------
    ma = np.array(const_xyz_to_lms)
    ma_inv = linalg.inv(ma)

    src_LMS = ma.dot(src)
    dst_LMS = ma.dot(dst)

    # M行列を求めよう
    # --------------------------------------
    mtx = [[dst_LMS[0]/src_LMS[0], 0.0, 0.0],
           [0.0, dst_LMS[1]/src_LMS[1], 0.0],
           [0.0, 0.0, dst_LMS[2]/src_LMS[2]]]

    m_mtx = ma_inv.dot(mtx).dot(ma)

    return m_mtx


def color_cvt(img, mtx):
    """
    # 概要
    img に対して mtx を適用する。
    # 注意事項
    例によって、RGBの並びを考えている。BGRの並びの場合は
    img[:, :, ::-1] してから関数をコールすること。
    """
    img_max = np.iinfo(img.dtype).max
    img_min = np.iinfo(img.dtype).min

    r, g, b = np.dsplit(img, 3)
    ro = r * mtx[0][0] + g * mtx[0][1] + b * mtx[0][2]
    go = r * mtx[1][0] + g * mtx[1][1] + b * mtx[1][2]
    bo = r * mtx[2][0] + g * mtx[2][1] + b * mtx[2][2]

    out_img = np.dstack((ro, go, bo))

    out_img[out_img < img_min] = img_min
    out_img[out_img > img_max] = img_max

    return out_img


def change_img_white_point(filename='wp04_1920x1080.jpg'):
    """
    # 概要
    色温度変換を行う。

    # 注意事項
    現状だと D65 --> D50 しか変換できない。
    気が向いたら任意の色温度に変換できるように拡張しよう。
    """
    img = cv2.imread(filename, cv2.IMREAD_ANYDEPTH | cv2.IMREAD_COLOR)
    m_mtx = get_white_point_conv_matrix()

    # XYZ 変換 Matrix と組み合わせて RGB 空間用の mtx を求める
    # ----------------------------------------------------
    rgb2xyz_mtx = get_rgb_to_xyz_matrix(gamut=const_sRGB_xy)
    xyz2rgb_mtx = linalg.inv(rgb2xyz_mtx)
    mtx = xyz2rgb_mtx.dot(m_mtx).dot(rgb2xyz_mtx)

    # 求めた mtx を使って画像を変換
    # ----------------------------------------------------
    img_out = color_cvt(img[:, :, ::-1], mtx)[:, :, ::-1]
    img_out = np.round(img_out).astype(img.dtype)

    # オリジナルと加工後の画像を比較表示
    # ------------------------------------
    fig = plt.figure(figsize=(10, 10))
    ax1 = fig.add_subplot(2, 1, 1)  # 引数はそれぞれ 縦数、横数、index
    ax2 = fig.add_subplot(2, 1, 2)
    ax1.imshow(img[:, :, ::-1])
    ax2.imshow(img_out[:, :, ::-1])

    root, ext = os.path.splitext(filename)
    out_filename = root + "_modify" + ext
    cv2.imwrite(out_filename, img_out)

    
change_img_white_point()

NameError: name 'get_rgb_to_xyz_matrix' is not defined